In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

import analysis as a

p = make_paths(a)

In [ ]:
w = pd.read_table(
    p['w_file_path'],
    index_col=0,
)

h = pd.read_table(
    p['h_file_path'],
    index_col=0,
)

In [ ]:
keep_threshold = 1.0

if keep_threshold < 1:

    feature_entropy = w.apply(
        ccal.compute_entropy,
        axis=1,
    )

    w = w.loc[feature_entropy.index[feature_entropy < feature_entropy.quantile(q=keep_threshold)]]

    print(w.shape)

    sample_entropy = h.apply(ccal.compute_entropy)

    h = h[sample_entropy.index[sample_entropy < sample_entropy.quantile(q=keep_threshold)]]

    print(h.shape)

In [ ]:
for i in (
    2007,
    725,
    90212,
    9909,
    2017,
    2018,
) + tuple(range(100)):

    nmf_map = ccal.NMFMap(
        w.T,
        h,
        mds_random_seed=i,
        wt_pull_power=a.NMF_MAP_WT_PULL_POWER,
        h_pull_power=a.NMF_MAP_H_PULL_POWER,
        plot=False,
    )

    for wt_or_h, hcc_k, bandwidth_factor, element_marker_size in (
        (
            'wt',
            a.WT_HCC_K,
            a.NMF_MAP_WT_BANDWIDTH_FACTOR,
            a.NMF_MAP_WT_ELEMENT_MARKER_SIZE,
        ),
        (
            'h',
            a.H_HCC_K,
            a.NMF_MAP_H_BANDWIDTH_FACTOR,
            a.NMF_MAP_H_ELEMENT_MARKER_SIZE,
        ),
    ):

        if wt_or_h == 'h':

            nmf_map.plot_nmf_map(
                wt_or_h,
                title=i,
                element_marker_size=element_marker_size,
            )

In [ ]:
wt_hcc__k_x_column_file_path = p['wt_hcc__k_x_column_file_path']

if os.path.isfile(wt_hcc__k_x_column_file_path):

    wt_hcc__k_x_column = pd.read_table(
        wt_hcc__k_x_column_file_path,
        index_col=0,
    )

    wt_element_states = wt_hcc__k_x_column.loc['K{}'.format(a.WT_HCC_K)]

    wt_element_states = wt_element_states[w.index]

else:

    wt_element_states = None

h_hcc__k_x_column_file_path = p['h_hcc__k_x_column_file_path']

if os.path.isfile(h_hcc__k_x_column_file_path):

    h_hcc__k_x_column = pd.read_table(
        h_hcc__k_x_column_file_path,
        index_col=0,
    )

    h_element_states = h_hcc__k_x_column.loc['K{}'.format(a.H_HCC_K)]

    h_element_states = h_element_states[h.columns]

else:

    h_element_states = None

In [ ]:
nmf_map = ccal.NMFMap(
    wt=w.T,
    h=h,
    mds_random_seed=a.NMF_MAP_MDS_RANDOM_SEED,
    wt_pull_power=a.NMF_MAP_WT_PULL_POWER,
    h_pull_power=a.NMF_MAP_H_PULL_POWER,
    plot=False,
)

In [ ]:
for wt_or_h, hcc_k, element_states, bandwidth_factor, element_marker_size in (
    (
        'wt',
        a.WT_HCC_K,
        wt_element_states,
        a.NMF_MAP_WT_BANDWIDTH_FACTOR,
        a.NMF_MAP_WT_ELEMENT_MARKER_SIZE,
    ),
    (
        'h',
        a.H_HCC_K,
        h_element_states, 
        a.NMF_MAP_H_BANDWIDTH_FACTOR,
        a.NMF_MAP_H_ELEMENT_MARKER_SIZE,
    ),
):

    nmf_map.plot_nmf_map(
        wt_or_h,
        title=a.NAME,
        element_marker_size=element_marker_size,
        html_file_path='{}/map.html'.format(p['{}_map_directory_path'.format(wt_or_h)]),
        plotly_file_path=p['{}_map_plotly_file_path'.format(wt_or_h)],
    )

    if element_states is not None:

        nmf_map.set_element_states(
            wt_or_h,
            element_states,
            bandwidth_factor=bandwidth_factor,
        )

        nmf_map.plot_nmf_map(
            wt_or_h,
            title=a.NAME,
            element_marker_size=element_marker_size,
            html_file_path='{}/map_{}.html'.format(
                p['{}_map_directory_path'.format(wt_or_h)],
                hcc_k,
            ),
            plotly_file_path=p['{}_state_map_plotly_file_path'.format(wt_or_h)],
        )

In [ ]:
ccal.dump_nmf_map(
    nmf_map,
    p['nmf_map_file_path'],
)

In [ ]:
feature_dicts = make_feature_dicts()

In [ ]:
for feature, feature_name in (
    (
        'mRNA',
        'ZEB1',
    ),
    (
        'mRNA',
        'AXL',
    ),
):

    nmf_map.plot_nmf_map(
        'h',
        annotation_x_element=feature_dicts[feature]['df'].loc[feature_name].to_frame().T,
        annotation_std_maxs=(3, ),
        title='({}) {}'.format(
            feature,
            feature_name,
        ),
        element_marker_size=a.NMF_MAP_H_ELEMENT_MARKER_SIZE,
        plotly_file_path='{}/{} {}'.format(
            p['{}_state_maps_plotly_directory_path'.format(wt_or_h)],
            feature,
            feature_name,
        ),
    )